### Dependencies:

In [1]:
!pip install -q -U transformers accelerate bitsandbytes
!pip install -q edge-tts pydub
!apt-get install -y ffmpeg

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import edge_tts
import asyncio
from pydub import AudioSegment
import io
import os

print("✅ Libraries installed! Make sure Internet is ON in settings.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 106.4 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 85.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

2025-12-13 12:52:59.904064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765630380.282809      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765630380.390138      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✅ Libraries installed! Make sure Internet is ON in settings.


### The Script Writer (LLM):

In [2]:
# 1. Setup Quantization (Make the model fit on free GPU)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model_id = "HuggingFaceH4/zephyr-7b-beta"

print(f"⏳ Loading {model_id} (this takes 2-3 mins)...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Create the text generation pipeline
script_writer = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024, # Long script
    temperature=0.7,     # Creative but not crazy
    top_p=0.95,
    repetition_penalty=1.15
)

print("✅ AI Writer is ready!")

⏳ Loading HuggingFaceH4/zephyr-7b-beta (this takes 2-3 mins)...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


✅ AI Writer is ready!


### The Script Generation Function:

In [3]:
def generate_script(topic):
    print(f"🤔 Brainstorming podcast about: {topic}...")
    
    prompt = [
        {
            "role": "system",
            "content": (
                "You are a professional podcast scriptwriter. Write a lively, engaging 2-minute conversation between two hosts, 'Alex' (energetic, curious) and 'Jamie' (calm, expert). "
                "They are discussing the user's topic. "
                "Keep it natural: use 'um', 'wow', 'exactly'. "
                "Format EXACTLY like this:\n"
                "Alex: [text]\n"
                "Jamie: [text]\n"
                "Alex: [text]\n"
                "Do not add stage directions like (laughs)."
            ),
        },
        {"role": "user", "content": f"Topic: {topic}"},
    ]
    
    # Format prompt for Zephyr
    prompt_formatted = tokenizer.apply_chat_template(prompt, tokenize=False, add_generation_prompt=True)
    
    outputs = script_writer(prompt_formatted)
    generated_text = outputs[0]["generated_text"]
    
    # Extract just the new script (remove the prompt)
    script_content = generated_text.split("<|assistant|>")[-1].strip()
    return script_content

# Test it quickly
test_script = generate_script("Why cats are actually aliens")
print("\n--- PREVIEW SCRIPT ---")
print(test_script[:500] + "...")

🤔 Brainstorming podcast about: Why cats are actually aliens...

--- PREVIEW SCRIPT ---
Alex: Hey Jamie, have you ever thought that our feline friends might be more than just ordinary house pets?

Jamie: (laughs) I don't know about that, Alex. Cats have been domesticated for centuries now. How could they possibly be aliens?

Alex: Well, think about it. They land on Earth from outer space through windows and doors without any parachutes or spacesuits. They can jump incredibly high and run faster than humans can run. Their eyes glow in the dark, and their fur seems to change colors b...


### The Audio Engine (TTS & Mixing):

In [4]:
async def generate_audio_segment(text, voice, filename):
    communicate = edge_tts.Communicate(text, voice)
    await communicate.save(filename)

async def create_podcast_audio(script_text):
    lines = script_text.strip().split('\n')
    combined_audio = AudioSegment.empty()
    
    print("🎙️ Recording Voice Actors...")
    
    temp_files = []
    
    for i, line in enumerate(lines):
        if not line.strip(): continue
        
        # Parse "Name: Text"
        if "Alex:" in line:
            speaker = "Alex"
            text = line.replace("Alex:", "").strip()
            voice = "en-US-GuyNeural" # Male voice
        elif "Jamie:" in line:
            speaker = "Jamie"
            text = line.replace("Jamie:", "").strip()
            voice = "en-US-JennyNeural" # Female voice
        else:
            continue # Skip lines that don't look like dialogue
            
        filename = f"line_{i}.mp3"
        await generate_audio_segment(text, voice, filename)
        
        # Load audio and append to track
        segment = AudioSegment.from_mp3(filename)
        combined_audio += segment
        
        # Add a tiny pause between speakers (300ms)
        combined_audio += AudioSegment.silent(duration=300)
        
        temp_files.append(filename)
        print(f"   Processed {speaker}: {text[:30]}...")

    # Cleanup temp files
    for f in temp_files:
        os.remove(f)
        
    return combined_audio

print("✅ Audio Studio is set up.")

✅ Audio Studio is set up.


## Action!

In [5]:
# --- SET YOUR TOPIC HERE ---
USER_TOPIC = "The future of Artificial Intelligence and robots taking over"

# 1. Generate Script
final_script = generate_script(USER_TOPIC)
print("\n📜 Full Script Generated:\n")
print(final_script)

# 2. Generate Audio (Async wrapper needed for Jupyter)
print("\n🎧 Synthesizing Audio (this might take 1-2 mins)...")
final_audio = await create_podcast_audio(final_script)

# 3. Save & Play
output_filename = "podcast_episode.mp3"
final_audio.export(output_filename, format="mp3")

print(f"\n✅ DONE! Podcast saved as '{output_filename}'")

# Play in Notebook
from IPython.display import Audio
Audio(output_filename)

🤔 Brainstorming podcast about: The future of Artificial Intelligence and robots taking over...

📜 Full Script Generated:

Alex: Hey Jamie, what’s up today? We have an exciting topic for our listeners - the future of AI and robots! Have you heard any recent developments in this field that you think could change the game?

Jamie: Absolutely, Alex. One thing that has really caught my attention is the progress being made in robotics. With advancements in sensors, actuators, and machine learning algorithms, robots are becoming more sophisticated than ever before. In fact, some experts predict that by the year 2050, we will see fully autonomous robots that can perform tasks as complex as driving cars or performing surgeries.

Alex: Wow, that sounds incredible! But do you think robots will eventually replace human jobs? I mean, we’ve all seen those movies where machines take over the world…

Jamie: That’s definitely a concern, but I believe that rather than replacing humans entirely, robots w